#### Update content of the CPM Atlas - generate maps based on the CPM data <br>
Author          : Team BETA<br>
Return Values   : png files<br>
Source data     : The data is preprocessed and provided by Tom Crocker from Met Office.

In [ ]:
import os
from pathlib import Path
from itertools import product
import numpy as np
import xarray as xr
# plotting
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as crs
from cartopy import feature as cfeature
# for clipping
import rioxarray

In [ ]:
# path to datasets
path_data = "/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Tom"

In [ ]:
# function for plotting maps
def plot_map(dataset, outfile, region, variable):
    # Plot
    fig = plt.figure(dpi=120)
    ax = fig.add_subplot(111, projection=crs.PlateCarree())
    if variable == "pr":
        levels = np.arange(-50, 51, 2)
        dataset.plot(ax=ax, levels=levels, cmap="BrBG",
                     cbar_kwargs={'label': ''})
    elif variable == "tas":
        levels = np.arange(-1, 5.1, 0.2)
        dataset.plot(ax=ax, levels=levels, cmap="YlOrRd",
                     cbar_kwargs={'label': ''})
    # colorbar
    # cbar = fig.colorbar(im)
    # cbar.set_label('')
    # Prettify
    ax.add_feature(cfeature.OCEAN, zorder=2)
    ax.coastlines(zorder=3)
    ax.axis('off')
    if variable == "pr":
        ax.set_title('Relative precipitation (%)')
    elif variable == "tas":
        ax.set_title('Relative temperature (degC)')

    # Save
    fig.savefig(outfile)
    plt.close()

def region_clip(infile, region, variable):
    # load dataset
    xds = xr.open_dataset(infile)
    # change filled values to 0, to avoid errors after clipping
    xds["unknown"] = xds.unknown.where(xds.unknown<1e+10, 0)
    # drop unused variables, otherwise it will cause 
    # an error when configuring the coordinate system
    xds = xds.drop_vars("lat_bnds")
    xds = xds.drop_vars("lon_bnds")
    # set-up the coordinate system known to rio
    xds.rio.write_crs("EPSG:4326", inplace=True)
    # clip data
    clipped_data = xds.unknown.rio.clip(regions[region], "EPSG:4326", all_touched=True)

    return clipped_data


In [ ]:
# define geometries for clipping by regions
regions = {
  "NW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-8.0, 40.4], [11.0, 40.4], [15.2, 58.6], [-12.5, 58.6], [-8.0, 40.4]]]
    }
  ],
  "SW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-10, 30], [7.4, 33], [5.7, 48.9], [-15, 45.4], [-10, 30]]]
    }
  ],
  "SE": [
    {
      'type': 'Polygon',
      'coordinates': [[[12.5, 34.3], [28.5, 34.3], [29.4, 40.9], [11.5, 40.9], [12.5, 34.3]]]
    }
  ],
  "C": [
    {
      'type': 'Polygon',
      'coordinates': [[[5.0, 44.5], [18.0, 45.5], [18.0, 56.0], [1.0, 53.0], [5.0, 44.5]]]
    }
  ],
  "CE": [
    {
      'type': 'Polygon',
      'coordinates': [[[17.8, 41.5], [31.3, 41.5], [32.8, 51.6], [16.4, 51.6], [17.8, 41.5]]]
    }
  ],
  "N": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 50.7], [26.7, 49.7], [44.1, 70.6], [-9.4, 72.6], [1, 50.7]]]
    }
  ],
  "AL": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 40], [17, 40], [17, 50], [1, 50], [1, 40]]]
    }
  ]
}

In [ ]:
# load all datasets and plot maps
seasons = ['DJF', 'MAM', 'JJA', 'SON']
variables = ['tas', 'pr']
#projects = ['CMIP5', 'CMIP6', 'CORDEX', 'cordex-cpm']

for variable, season, region in product(variables, seasons, regions.keys()):
    directory = path_data + f"/{variable}_anoms/main/{season}"
    for infile in Path(directory).iterdir():
        # handle exceptions of file naming
        if 'rg' in str(infile):
            project, model, _, _, season = infile.stem.split('_')
            model = model + '-rg' # include the 'rg' tag in the name
        elif 'drivers' in str(infile):
            pass
        else:
            project, model, _, season = infile.stem.split('_')
            # handle exceptions of redundant region file naming
            if project == "cordex-cpm":
                try:
                    model, redundant_region = model.split(' ')
                except:
                    pass 
        # to handle the exception when the clipped region is empty
        try:
           clipped_region = region_clip(infile, region, variable)
           # skip regions with only a few valid points
           if (len(np.unique(clipped_region.data)) < 50):
               pass
           else:
               outfile = f"../static/maps/{region}/{variable}/{project}_{model}_{season}.png"
               plot_map(clipped_region, outfile, region, variable)
               print(f"Processed {infile}, created {outfile}.")
        except Exception as e:
            print(e)
            pass